In [ ]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

from utils.dataset_init_utils import get_dataset, get_dl_test_set
from utils.pytorchtools import EarlyStopping
from model.slvit_ae import *
from model.hybrid_vqvae import *
import datetime

config = {'patch_sizes_idx': 2, 'h1': 16, 'h2': 16, 'h3': 16, 'hidden_dim': 64, 'batch_size': 25,
                      'mode': 'linear', 'lr': 0.0020005604375492943,
                      'beta': 0.19675492940423106, 'decay': 0.9671892237064758, 'num_embeddings': 64, 'embeddings_dim':512
                      }

def transform_to_tensor(x, device):
    x = torch.from_numpy(x)
    return x.to(device)

def output(model, data_loader):
    model.eval()
    with torch.no_grad():
        i = 0
        for data, target in data_loader:
            if i == 0:
                pre = model(data)[0]
                true = target
            else:
                pre = torch.cat((pre, model(data)[0]), dim=0)
                true = torch.cat((true, target), dim=0)
            i += 1
        return true, pre

def evaluate(model, data_loader):
    true, pre = output(model, data_loader)

    nmse, mse, nmse_all = nmse_eval(true, pre)

    print('\nvalidation loss (mse): ' + format(mse.item(), '.3f'))
    print('validation loss (nmse): ' + format(nmse.item(), '.3f'))
    print('----------------------------------------------------\n')
    return nmse, mse



def nmse_eval(y_true, y_pred):
    y_true = y_true.reshape(len(y_true), -1)
    y_pred = y_pred.reshape(len(y_pred), -1)

    mse = torch.sum(abs(y_pred - y_true) ** 2, dim=1)
    power = torch.sum(abs(y_true) ** 2, dim=1)

    nmse = mse / power
    return torch.mean(nmse), torch.mean(mse), nmse


In [ ]:
device = "cuda:0"

# load data
x_train, x_val, x_test = get_dataset(
    direction='uplink', scenario_name='mimo_umi_nlos', gap='_')
x_test_dl_1 = get_dl_test_set(scenario_name='mimo_umi_nlos', gap='1')

x_train = transform_to_tensor(x_train, device)
x_val = transform_to_tensor(x_val, device)
x_test = transform_to_tensor(x_test, device)

x_test_dl_1 = transform_to_tensor(x_test_dl_1, device)

# create dataloader
train_data_set = TensorDataset(x_train, x_train)
val_data_set = TensorDataset(x_val, x_val)
test_data_set = TensorDataset(x_test, x_test)

dl_test_set_1 = TensorDataset(x_test_dl_1, x_test_dl_1)

train_loader = DataLoader(
    train_data_set, batch_size=config["batch_size"], shuffle=True)
validation_loader = DataLoader(
    val_data_set, batch_size=1000, shuffle=False)
test_loader = DataLoader(test_data_set, batch_size=1000, shuffle=False)

dl_test_loader_1 = DataLoader(
    dl_test_set_1, batch_size=1000, shuffle=False)

patch_size_combinations = [
    [8, 8],
    [4, 16],
    [2, 32],
    [1, 64],
    [2, 64],
    [4, 32],
    [8, 16],
    [4, 8],
    [2, 16],
    [1, 32]
]

idx = config["patch_sizes_idx"]

p1 = patch_size_combinations[idx][0]
p2 = patch_size_combinations[idx][1]

now = datetime.datetime.now()
foldername = 'resVQVAE_' + now.strftime("%Y-%m-%d %H-%M-%S")

   #for j, key in enumerate(config.keys()):
   #    foldername += f'{key}_{config[key]}_'

if not os.path.exists(foldername):
        os.mkdir(foldername)

model = NDVitResVAE(
        dims_enc=[128, 64, 32],
        heads_enc=[config["h1"], config["h2"], config["h3"]],
        overall_dim=2048,
        p1=p1,
        p2=p2,
        hidden_dim=config["hidden_dim"],
        mode=config["mode"],
        num_embeddings=config["num_embeddings"],
        embedding_dim=config["embeddings_dim"],
        succ_prob=1/config['embeddings_dim'],
        decay=config["decay"],
        beta=0.25
    ).to(device=device)


In [ ]:
model_dict=torch.load('resVQVAE_2023-09-30 17-38-36/checkpoint.pt')
model.load_state_dict(model_dict)
model.eval()

In [ ]:
true_channels, pred_channels = output(model, test_loader)
nmse_test, mse_test, nmse_ul = nmse_eval(true_channels, pred_channels)
true_channels, pred_channels = output(model, dl_test_loader_1)
nmse_test, mse_test, nmse_dl_1 = nmse_eval(true_channels, pred_channels)

In [ ]:
nmseul_log = 10*np.log10(nmse_ul.cpu())

# getting data of the histogram
count, bins_count = np.histogram(nmseul_log, bins=50)
  
# finding the PDF of the histogram using count values
pdf = count / sum(count)
  
# using numpy np.cumsum to calculate the CDF
# We can also find using the PDF values by looping and adding
cdf = np.cumsum(pdf)
  
# plotting PDF and CDF
#plt.plot(bins_count[1:], pdf, color="red", label="PDF")
plt.plot(bins_count[1:], cdf, label="CDF")
plt.legend()

In [11]:
import torch
import torch.nn as nn
import scipy.io as scio
import h5py
import numpy as np

In [17]:
data = h5py.File(r'E:\\Code_shaohua\\datasets\\qdg_umi5g_7p68\\val.mat')

print(data.keys())
print(data.values())
train = np.array(data['val'])

<KeysViewHDF5 ['val']>
ValuesViewHDF5(<HDF5 file "val.mat" (mode r)>)


In [15]:
print(train.shape)

(30000, 32, 64)


In [16]:
data.

TypeError: 'KeysViewHDF5' object is not subscriptable